In [1]:
# python 3.6
# tensorflow 2.3.1 cpu
# sklearn.__version__ == 0.23.2

import pandas as pd
import numpy as np
import pickle 
from time import time

from sklearn.utils import shuffle # shuffle打乱样本的顺序，它只会打乱样本的顺序，每个样本的数据维持不变。
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def save_obj(obj, file):
    with open(file, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(file ):
    with open(file, 'rb') as f:
        return pickle.load(f)


In [2]:
# 评价指标函数
def f1_score_get(precision, recall):
    # tf.keras.backend.epsilon() 的值为 1e-07
    # 1.0e-7 == tf.keras.backend.epsilon() 得到 True
    f1 = 2 * precision * recall/( precision + recall + 1.0e-7 )
    return f1


# valid_y:真实标签
# predict_y:预测标签
def eval_p_r_f1(valid_y, predict_y):
    # precision 0 1 2 3
    # setting labels=[pos_label] and average != 'binary' will report scores for that label only.
    accuracy = accuracy_score(
                    y_true = valid_y,
                    y_pred = predict_y
                              )
    
    precision_0 = precision_score(
                    y_true = valid_y,
                    y_pred = predict_y,
                    labels = [0],
                    pos_label = 0,
                    average = 'micro'
                    )

    precision_1 = precision_score(
                    y_true = valid_y,
                    y_pred = predict_y,
                    labels = [1],
                    pos_label = 1,
                    average = 'micro'
                    )

    precision_2 = precision_score(
                    y_true = valid_y,
                    y_pred = predict_y,
                    labels = [2],
                    pos_label = 2,
                    average = 'micro'
                    )

    precision_3 = precision_score(
                    y_true = valid_y,
                    y_pred = predict_y,
                    labels = [3],
                    pos_label = 3,
                    average = 'micro'
                    )
    # recall 0 1 2 3
    recall_0 = recall_score(
                    y_true = valid_y,
                    y_pred = predict_y,
                    labels = [0],
                    pos_label = 0,
                    average = 'micro'
                    )

    recall_1 = recall_score(
                    y_true = valid_y,
                    y_pred = predict_y,
                    labels = [1],
                    pos_label = 1,
                    average = 'micro'
                    )

    recall_2 = recall_score(
                    y_true = valid_y,
                    y_pred = predict_y,
                    labels = [2],
                    pos_label = 2,
                    average = 'micro'
                    )

    recall_3 = recall_score(
                    y_true = valid_y,
                    y_pred = predict_y,
                    labels = [3],
                    pos_label = 3,
                    average = 'micro'
                    )
    
    # f1_score 0 1 2 3
    f1_score_0 = f1_score_get( precision_0, recall_0 )
    f1_score_1 = f1_score_get( precision_1, recall_1 )
    f1_score_2 = f1_score_get( precision_2, recall_2 )
    f1_score_3 = f1_score_get( precision_3, recall_3 )
    
    #由y_true 计算各标签权重
    num_0 = np.sum(valid_y==0)
    num_1 = np.sum(valid_y==1)
    num_2 = np.sum(valid_y==2)
    num_3 = np.sum(valid_y==3)
    
    total = num_0 + num_1 + num_2 + num_3
    p_0 = num_0/total
    p_1 = num_1/total
    p_2 = num_2/total
    p_3 = num_3/total
    
    precision_avg = p_0 * precision_0 + p_1 * precision_1 + p_2 * precision_2 + p_3 * precision_3
    recall_avg = p_0 * recall_0 + p_1 * recall_1 + p_2 * recall_2 + p_3 * recall_3
    f1_score_avg = p_0 * f1_score_0 + p_1 * f1_score_1 + p_2 * f1_score_2 + p_3 * f1_score_3
    
    return accuracy, precision_avg, recall_avg, f1_score_avg


In [3]:
# train = pd.read_csv('./data/train_location.csv', usecols=['content', 'label'])
# valid = pd.read_csv('./data/valid_location.csv', usecols=['content', 'label'])

# train = pd.read_csv('./data/train_service.csv', usecols=['content', 'label'])
# valid = pd.read_csv('./data/valid_service.csv', usecols=['content', 'label'])

# train = pd.read_csv('./data/train_price.csv', usecols=['content', 'label'])
# valid = pd.read_csv('./data/valid_price.csv', usecols=['content', 'label'])

# train = pd.read_csv('./data/train_environment.csv', usecols=['content', 'label'])
# valid = pd.read_csv('./data/valid_environment.csv', usecols=['content', 'label'])

train = pd.read_csv( './data/train_dish.csv', usecols=['content', 'label'])
valid = pd.read_csv( './data/valid_dish.csv', usecols=['content', 'label'])

train = shuffle(train, random_state = 2020)
valid = shuffle(valid, random_state = 2020)

train = shuffle(train, random_state = 42)
valid = shuffle(valid, random_state = 42)

print(train.shape)
print(valid.shape)


(105000, 2)
(15000, 2)


In [4]:
# 读取 X
# 不同aspect，文本数据一样，不同体现在标签上

train_x = train.content.values.tolist()
valid_x = valid.content.values.tolist()

print(train_x[0:3])

[' 大众点评 十月份 VIP 会员 聚餐 十几个 兴趣相投 吃货们 点评 聚在 周日 晚上 万家 一炉 划分 音乐餐厅 自助餐厅 两个 区域 装修 看图片 点评 菜品 十七个 菜 只说 我爱吃 先赞 农家 扒茄条 口感鲜香 入口微甜 老少咸宜 必点 菜品 鲍汁 一品豆腐 入味 不油腻 美味 金丝虾球 也很 不错 土豆丝 炸的 酥脆 小朋友 肯定 喜欢 我还 喜欢 七彩鱼 皮 越往 吃芥末 越冲 过瘾 米粉肉 甜而不腻 美味 田七 爽口凉菜 酸汤肥牛 … 量少 … 吃了 忘记 味道 … 说 的是 音乐餐厅 八点 点歌 爱唱歌 的你 千万不要错过 媲美 专业音响 … ', ' 第一次 抽中 同城 试吃活动 很开心 ✌ ️ 听说 小伙伴 都是 老会员 参加 活动 好有爱 ❤ ️ 这家店 位置 不是很好 找 到了 饭店 没什么 顾客 地理位置 劣势 原因 服务员态度 搞错 家的 菜名 尴尬 招牌菜 金椒脆鱼 锅 每个人 习惯 鱼 口感 可以接受 还有一个 猪脚 虾 很好吃 猪脚 炖 入味 凉拌 蕨根粉 入味 酸辣粉 味道 店里 凉拌菜 觉 居多 凉拌菜 各类 肉品 锅 火锅 会吃 过多 拒绝 店家 提议 火锅 榴莲飞饼 味道 很好 榴莲肉 多点 更好 说 点的 口水鸡 味道 也很 鸡肉 上夹 肥 油太多 腻味 桂花糖莲藕 江浙菜馆 做的好 凉拌菜 川菜馆 特别 感受 果汁 兑水 兑多 没什么味道 店家 可以增加 炒菜 顾客 多点 选择 ^ _ ^ ', ' 同事 阅兵 小长假 美好 广场 吃的 说是 好吃 下班 直奔 旗舰店 想着 肯定 好吃 点菜 妹纸 讲 不出 道道 问 好吃 说 没吃过 上个 点评 评价点 上菜 搞不清 桌球 错了 想着 价位 比某 底捞 便宜 算了 人也 不容易 锅底 开吃 刚涮 根干 吧吧 鸭肠 妹纸 加汤 可乐 来瓶 茶杯 喝了 上了 三 玻璃杯 华丽丽 的囧 饮料杯 干嘛 早上 服务 无力吐槽 菜品 鸭肠 挂 架子 上的 看着 甩 洋气 表面 干 一夹 长条 缠 架子 自己动手 放 盘子 里 夹 琥珀 肉 口感 还能 吃到 筋 原谅我 牙口 不好 鲜毛肚 也是醉了 黑黑白白 都有 安慰 极品毛肚 求 新鲜 就好 天下 宽粉 味道还行 QQ 弹弹 极品 五花肉 真的很 极品 白白的 肥肉 无从 下口 首相 丸子 姜 喜欢 吃姜 这是 不愉快 拔

In [5]:
# 读取 Y
# 标签
train_y = train.label.values+2
valid_y = valid.label.values+2

In [6]:
n_features = 1000

tfidf_vectorizer = TfidfVectorizer(
                                        max_df=0.95, 
                                        min_df=2,
                                        max_features=n_features,
                                   )
corpus = train_x + valid_x
print(len(corpus))

t0 = time()
tfidf = tfidf_vectorizer.fit(corpus)

120000


In [7]:
print("done in %0.3fs." % (time() - t0))
train_x = tfidf.transform(train_x)
valid_x = tfidf.transform(valid_x)

done in 9.081s.


In [8]:
print(train_x.shape)
print(valid_x.shape)

(105000, 1000)
(15000, 1000)


In [9]:
# 标准化
scaler = StandardScaler(with_mean=False)
train_x = scaler.fit_transform(train_x)
valid_x = scaler.fit_transform(valid_x)

In [ ]:
'''
class sklearn.ensemble.RandomForestClassifier(
    n_estimators=100, 
    *, 
    criterion='gini', 
    max_depth=None, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features='auto',  #“sqrt”
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    min_impurity_split=None, 
    bootstrap=True, #True
    oob_score=False, 
    n_jobs=None, 
    random_state=None, # 2020
    verbose=0, # 1
    warm_start=False, 
    class_weight=None, # “balanced”
    ccp_alpha=0.0, # ?
    max_samples=None
    )
'''


In [10]:
n_estimators_range = [1000, 5000, 10000]
max_depth_range = [10,20,50]
min_samples_split_range = [2, 10, 50]
min_samples_leaf_range = [1, 5, 10, 20, 50]

In [11]:
for n_estimator in n_estimators_range:
    for max_depth in max_depth_range:
        for min_samples_split in min_samples_split_range:
            for min_samples_leaf in min_samples_leaf_range:
                print('n_estimator = %d' % n_estimator)
                print('max_depth = %d' % max_depth)
                print('min_samples_split = %d' % min_samples_split)
                print('min_samples_leaf = %d' % min_samples_leaf)
                clf = RandomForestClassifier(
                                                n_estimators= n_estimator,
                                                criterion='gini',
                                                max_depth= max_depth,
                                                min_samples_split = min_samples_split,
                                                min_samples_leaf = min_samples_leaf,
                                                max_features= 'sqrt',
                                                bootstrap=True,
                                                random_state= 2020,
                                                verbose= 1,
                                                class_weight= 'balanced',
                                                n_jobs = -1
                                            )
                t0 = time()
                clf.fit(train_x, train_y)
                print("done in %0.3fs." % (time() - t0))
                predict_y = clf.predict(valid_x)
                # predict_prob_y = clf.predict_proba(valid_x)
                # 评价指标值
                accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
                print('accuracy：%0.5f' % accuracy)
                print('precision_avg: %0.5f' % precision_avg)
                print('recall_avg: %0.5f' % recall_avg)
                print('f1_score_avg: %0.5f' % f1_score_avg)
                
                

n_estimator = 1000
max_depth = 10
min_samples_split = 2
min_samples_leaf = 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.0s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.1s


done in 8.476s.


[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed:    0.3s finished


accuracy：0.74380
precision_avg: 0.79753
recall_avg: 0.74380
f1_score_avg: 0.76212
n_estimator = 1000
max_depth = 10
min_samples_split = 2
min_samples_leaf = 5


...

accuracy：0.78627
precision_avg: 0.78041
recall_avg: 0.78627
f1_score_avg: 0.77857
n_estimator = 5000
max_depth = 50
min_samples_split = 2
min_samples_leaf = 10


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:   48.3s
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  5.2min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s


done in 314.718s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.8s


KeyboardInterrupt: 

In [11]:
n_estimators_range = [5000, 10000]
max_depth_range = [10,20,50]
min_samples_split_range = [2, 10, 50]
min_samples_leaf_range = [1, 5, 10, 20, 50]

In [12]:
##### 之前参数设置不合理，改一下


for n_estimator in n_estimators_range:
    for max_depth in max_depth_range:
        for min_samples_split in min_samples_split_range:
            for min_samples_leaf in min_samples_leaf_range:
                # 设置这个条件才合理
                if min_samples_split > min_samples_leaf:
                        print('n_estimator = %d' % n_estimator)
                        print('max_depth = %d' % max_depth)
                        print('min_samples_split = %d' % min_samples_split)
                        print('min_samples_leaf = %d' % min_samples_leaf)
                        clf = RandomForestClassifier(
                                                        n_estimators= n_estimator,
                                                        criterion='gini',
                                                        max_depth= max_depth,
                                                        min_samples_split = min_samples_split,
                                                        min_samples_leaf = min_samples_leaf,
                                                        max_features= 'sqrt',
                                                        bootstrap=True,
                                                        random_state= 2020,
                                                        verbose= 1,
                                                        class_weight= 'balanced',
                                                        n_jobs = -1
                                                    )
                        t0 = time()
                        clf.fit(train_x, train_y)
                        print("done in %0.3fs." % (time() - t0))
                        predict_y = clf.predict(valid_x)
                        # predict_prob_y = clf.predict_proba(valid_x)
                        # 评价指标值
                        accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
                        print('accuracy：%0.5f' % accuracy)
                        print('precision_avg: %0.5f' % precision_avg)
                        print('recall_avg: %0.5f' % recall_avg)
                        print('f1_score_avg: %0.5f' % f1_score_avg)
                
                

n_estimator = 5000
max_depth = 10
min_samples_split = 2
min_samples_leaf = 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:   41.4s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s


done in 43.400s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 5000 out of 5000 | elapsed:    2.1s finished


accuracy：0.73920
precision_avg: 0.79758
recall_avg: 0.73920
f1_score_avg: 0.75956
n_estimator = 5000
max_depth = 10
min_samples_split = 10
min_samples_leaf = 1


...

accuracy：0.76787
precision_avg: 0.78259
recall_avg: 0.76787
f1_score_avg: 0.77279
n_estimator = 10000
max_depth = 50
min_samples_split = 50
min_samples_leaf = 20


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 6010 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 7160 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 8410 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 9760 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 10000 out of

done in 454.547s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 6010 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done 7160 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 8410 tasks      | elapsed:    3.8s
[Parallel(n_jobs=20)]: Done 9760 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done 10000 out of 10000 | elapsed:    4.5s finished


accuracy：0.74880
precision_avg: 0.79181
recall_avg: 0.74880
f1_score_avg: 0.76576


In [20]:
predict_y = clf.predict(valid_x)
predict_prob_y = clf.predict_proba(valid_x)

In [23]:
# 评价指标值
precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)

print('precision_avg: %0.5f' % precision_avg)
print('recall_avg: %0.5f' % recall_avg)
print('f1_score_avg: %0.5f' % f1_score_avg)

precision_avg: 0.81552
recall_avg: 0.69727
f1_score_avg: 0.73461


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 2) is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 3) is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [10]:
clf = RandomForestClassifier(
                                n_estimators= 5000,
                                criterion='gini',
                                max_depth= 50,
                                min_samples_split = 50,
                                min_samples_leaf = 1,
                                max_features= 'sqrt',
                                bootstrap=True,
                                random_state= 2020,
                                verbose= 1,
                                class_weight= 'balanced',
                                n_jobs = -1
                            )
t0 = time()
clf.fit(train_x, train_y)
print("done in %0.3fs." % (time() - t0))
predict_y = clf.predict(valid_x)
# predict_prob_y = clf.predict_proba(valid_x)
# 评价指标值
accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
print('accuracy：%0.5f' % accuracy )
print('precision_avg: %0.5f' % precision_avg )
print('recall_avg: %0.5f' % recall_avg )
print('f1_score_avg: %0.5f' % f1_score_avg )


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 390 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 740 tasks      | elapsed:   45.3s
[Parallel(n_jobs=-1)]: Done 1190 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1740 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2390 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 3140 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 3990 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 4940 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  5.0min finished
[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 140 tasks      | elapsed:    0.1s
[Parallel(n_jobs=30)]: Done 390 tasks      | elapsed:    0.2s


done in 302.388s.


[Parallel(n_jobs=30)]: Done 740 tasks      | elapsed:    0.3s
[Parallel(n_jobs=30)]: Done 1190 tasks      | elapsed:    0.5s
[Parallel(n_jobs=30)]: Done 1740 tasks      | elapsed:    0.7s
[Parallel(n_jobs=30)]: Done 2390 tasks      | elapsed:    0.9s
[Parallel(n_jobs=30)]: Done 3140 tasks      | elapsed:    1.2s
[Parallel(n_jobs=30)]: Done 3990 tasks      | elapsed:    1.5s


accuracy：0.63520
precision_avg: 0.62039
recall_avg: 0.63520
f1_score_avg: 0.62224


[Parallel(n_jobs=30)]: Done 4940 tasks      | elapsed:    1.9s
[Parallel(n_jobs=30)]: Done 5000 out of 5000 | elapsed:    1.9s finished


In [28]:
clf = RandomForestClassifier(
                                n_estimators= 5000,
                                criterion='gini',
                                max_depth= 50,
                                min_samples_split = 50,
                                min_samples_leaf = 1,
                                max_features= 'sqrt',
                                bootstrap=True,
                                random_state= 2020,
                                verbose= 1,
                                class_weight= 'balanced',
                                n_jobs = -1
                            )
t0 = time()
clf.fit(train_x, train_y)
print("done in %0.3fs." % (time() - t0))
predict_y = clf.predict(valid_x)
# predict_prob_y = clf.predict_proba(valid_x)
# 评价指标值
accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
print('accuracy：%0.5f' % accuracy)
print('precision_avg: %0.5f' % precision_avg)
print('recall_avg: %0.5f' % recall_avg)
print('f1_score_avg: %0.5f' % f1_score_avg)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  6.7min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s
[Parallel(n_j

done in 402.268s.


[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.3s


accuracy：0.60747
precision_avg: 0.60396
recall_avg: 0.60747
f1_score_avg: 0.59642


[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 5000 out of 5000 | elapsed:    1.5s finished


In [10]:
clf = RandomForestClassifier(
                                n_estimators= 5000,
                                criterion='gini',
                                max_depth= 50,
                                min_samples_split = 50,
                                min_samples_leaf = 1,
                                max_features= 'sqrt',
                                bootstrap=True,
                                random_state= 2020,
                                verbose= 1,
                                class_weight= 'balanced',
                                n_jobs = -1
                            )
t0 = time()
clf.fit(train_x, train_y)
print("done in %0.3fs." % (time() - t0))
predict_y = clf.predict(valid_x)
# predict_prob_y = clf.predict_proba(valid_x)
# 评价指标值
accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
print('accuracy：%0.5f' % accuracy)
print('precision_avg: %0.5f' % precision_avg)
print('recall_avg: %0.5f' % recall_avg)
print('f1_score_avg: %0.5f' % f1_score_avg)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   36.3s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  7.2min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s
[Parallel(n_j

done in 433.064s.


[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.5s


accuracy：0.68453
precision_avg: 0.65330
recall_avg: 0.68453
f1_score_avg: 0.65628


[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 5000 out of 5000 | elapsed:    1.9s finished


In [18]:
clf = RandomForestClassifier(
                                n_estimators= 5000,
                                criterion='gini',
                                max_depth= 50,
                                min_samples_split = 50,
                                min_samples_leaf = 1,
                                max_features= 'sqrt',
                                bootstrap=True,
                                random_state= 2020,
                                verbose= 1,
                                class_weight= 'balanced',
                                n_jobs = -1
                            )
t0 = time()
clf.fit(train_x, train_y)
print("done in %0.3fs." % (time() - t0))
predict_y = clf.predict(valid_x)
# predict_prob_y = clf.predict_proba(valid_x)
# 评价指标值
accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
print('accuracy：%0.5f' % accuracy)
print('precision_avg: %0.5f' % precision_avg)
print('recall_avg: %0.5f' % recall_avg)
print('f1_score_avg: %0.5f' % f1_score_avg)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  8.3min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s


done in 499.574s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.4s


accuracy：0.65360
precision_avg: 0.65305
recall_avg: 0.65360
f1_score_avg: 0.64858


[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 5000 out of 5000 | elapsed:    1.8s finished


In [10]:
n_estimators_range = [1000, 5000, 10000]
max_depth_range = [10,20,50]
min_samples_split_range = [2, 10, 50]
min_samples_leaf_range = [1, 5, 10, 20, 50]


for n_estimator in n_estimators_range:
    for max_depth in max_depth_range:
        for min_samples_split in min_samples_split_range:
            for min_samples_leaf in min_samples_leaf_range:
                # 设置这个条件才合理
                if min_samples_split > min_samples_leaf:
                        print('n_estimator = %d' % n_estimator)
                        print('max_depth = %d' % max_depth)
                        print('min_samples_split = %d' % min_samples_split)
                        print('min_samples_leaf = %d' % min_samples_leaf)
                        clf = RandomForestClassifier(
                                                        n_estimators= n_estimator,
                                                        criterion='gini',
                                                        max_depth= max_depth,
                                                        min_samples_split = min_samples_split,
                                                        min_samples_leaf = min_samples_leaf,
                                                        max_features= 'sqrt',
                                                        bootstrap=True,
                                                        random_state= 2020,
                                                        verbose= 1,
                                                        class_weight= 'balanced',
                                                        n_jobs = -1
                                                    )
                        t0 = time()
                        clf.fit(train_x, train_y)
                        print("done in %0.3fs." % (time() - t0))
                        predict_y = clf.predict(valid_x)
                        # predict_prob_y = clf.predict_proba(valid_x)
                        # 评价指标值
                        accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
                        print('accuracy：%0.5f' % accuracy)
                        print('precision_avg: %0.5f' % precision_avg)
                        print('recall_avg: %0.5f' % recall_avg)
                        print('f1_score_avg: %0.5f' % f1_score_avg)
                
                

n_estimator = 1000
max_depth = 10
min_samples_split = 2
min_samples_leaf = 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.7s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s


done in 8.236s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed:    0.4s finished


accuracy：0.58093
precision_avg: 0.60521
recall_avg: 0.58093
f1_score_avg: 0.56988
n_estimator = 1000
max_depth = 10
min_samples_split = 10
min_samples_leaf = 1


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 6010 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 7160 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 8410 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 9760 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 10000 out of

done in 419.938s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 6010 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 7160 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 8410 tasks      | elapsed:    2.7s


accuracy：0.61140
precision_avg: 0.62779
recall_avg: 0.61140
f1_score_avg: 0.60897


[Parallel(n_jobs=20)]: Done 9760 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 10000 out of 10000 | elapsed:    3.1s finished


In [10]:
n_estimators_range = [1000, 5000, 10000]
max_depth_range = [10,20,50]
min_samples_split_range = [2, 10, 50]
min_samples_leaf_range = [1, 5, 10, 20, 50]


for n_estimator in n_estimators_range:
    for max_depth in max_depth_range:
        for min_samples_split in min_samples_split_range:
            for min_samples_leaf in min_samples_leaf_range:
                # 设置这个条件才合理
                if min_samples_split > min_samples_leaf:
                        print('n_estimator = %d' % n_estimator)
                        print('max_depth = %d' % max_depth)
                        print('min_samples_split = %d' % min_samples_split)
                        print('min_samples_leaf = %d' % min_samples_leaf)
                        clf = RandomForestClassifier(
                                                        n_estimators= n_estimator,
                                                        criterion='gini',
                                                        max_depth= max_depth,
                                                        min_samples_split = min_samples_split,
                                                        min_samples_leaf = min_samples_leaf,
                                                        max_features= 'sqrt',
                                                        bootstrap=True,
                                                        random_state= 2020,
                                                        verbose= 1,
                                                        class_weight= 'balanced',
                                                        n_jobs = -1
                                                    )
                        t0 = time()
                        clf.fit(train_x, train_y)
                        print("done in %0.3fs." % (time() - t0))
                        predict_y = clf.predict(valid_x)
                        # predict_prob_y = clf.predict_proba(valid_x)
                        # 评价指标值
                        accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
                        print('accuracy：%0.5f' % accuracy)
                        print('precision_avg: %0.5f' % precision_avg)
                        print('recall_avg: %0.5f' % recall_avg)
                        print('f1_score_avg: %0.5f' % f1_score_avg)
                
                

n_estimator = 1000
max_depth = 10
min_samples_split = 2
min_samples_leaf = 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.3s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s


done in 7.732s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed:    0.4s finished


accuracy：0.57320
precision_avg: 0.59827
recall_avg: 0.57320
f1_score_avg: 0.55731
n_estimator = 1000
max_depth = 10
min_samples_split = 10
min_samples_leaf = 1


...

accuracy：0.60180
precision_avg: 0.60797
recall_avg: 0.60180
f1_score_avg: 0.59307
n_estimator = 10000
max_depth = 50
min_samples_split = 50
min_samples_leaf = 20


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:   48.7s
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 6010 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 7160 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 8410 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 9760 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 10000 out of

done in 401.104s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 6010 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 7160 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done 8410 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 9760 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 10000 out of 10000 | elapsed:    3.6s finished


accuracy：0.59867
precision_avg: 0.60944
recall_avg: 0.59867
f1_score_avg: 0.59054


In [10]:
n_estimators_range = [1000, 5000, 10000]
max_depth_range = [10,20,50]
min_samples_split_range = [2, 10, 50]
min_samples_leaf_range = [1, 5, 10, 20, 50]


for n_estimator in n_estimators_range:
    for max_depth in max_depth_range:
        for min_samples_split in min_samples_split_range:
            for min_samples_leaf in min_samples_leaf_range:
                # 设置这个条件才合理
                if min_samples_split > min_samples_leaf:
                        print('n_estimator = %d' % n_estimator)
                        print('max_depth = %d' % max_depth)
                        print('min_samples_split = %d' % min_samples_split)
                        print('min_samples_leaf = %d' % min_samples_leaf)
                        clf = RandomForestClassifier(
                                                        n_estimators= n_estimator,
                                                        criterion='gini',
                                                        max_depth= max_depth,
                                                        min_samples_split = min_samples_split,
                                                        min_samples_leaf = min_samples_leaf,
                                                        max_features= 'sqrt',
                                                        bootstrap=True,
                                                        random_state= 2020,
                                                        verbose= 1,
                                                        class_weight= 'balanced',
                                                        n_jobs = -1
                                                    )
                        t0 = time()
                        clf.fit(train_x, train_y)
                        print("done in %0.3fs." % (time() - t0))
                        predict_y = clf.predict(valid_x)
                        # predict_prob_y = clf.predict_proba(valid_x)
                        # 评价指标值
                        accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
                        print('accuracy：%0.5f' % accuracy)
                        print('precision_avg: %0.5f' % precision_avg)
                        print('recall_avg: %0.5f' % recall_avg)
                        print('f1_score_avg: %0.5f' % f1_score_avg)
                
                

n_estimator = 1000
max_depth = 10
min_samples_split = 2
min_samples_leaf = 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.3s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s


done in 7.720s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed:    0.4s finished


accuracy：0.62940
precision_avg: 0.66037
recall_avg: 0.62940
f1_score_avg: 0.62863
n_estimator = 1000
max_depth = 10
min_samples_split = 10
min_samples_leaf = 1


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:   50.4s
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 6010 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 7160 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 8410 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 9760 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 10000 out of

done in 415.377s.


[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 6010 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 7160 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 8410 tasks      | elapsed:    2.2s


accuracy：0.65340
precision_avg: 0.66487
recall_avg: 0.65340
f1_score_avg: 0.65109


[Parallel(n_jobs=20)]: Done 9760 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done 10000 out of 10000 | elapsed:    2.7s finished


In [10]:
n_estimators_range = [1000, 5000, 10000]
max_depth_range = [10,20,50]
min_samples_split_range = [2, 10, 50]
min_samples_leaf_range = [1, 5, 10, 20, 50]


for n_estimator in n_estimators_range:
    for max_depth in max_depth_range:
        for min_samples_split in min_samples_split_range:
            for min_samples_leaf in min_samples_leaf_range:
                # 设置这个条件才合理
                if min_samples_split > min_samples_leaf:
                        print('n_estimator = %d' % n_estimator)
                        print('max_depth = %d' % max_depth)
                        print('min_samples_split = %d' % min_samples_split)
                        print('min_samples_leaf = %d' % min_samples_leaf)
                        clf = RandomForestClassifier(
                                                        n_estimators= n_estimator,
                                                        criterion='gini',
                                                        max_depth= max_depth,
                                                        min_samples_split = min_samples_split,
                                                        min_samples_leaf = min_samples_leaf,
                                                        max_features= 'sqrt',
                                                        bootstrap=True,
                                                        random_state= 2020,
                                                        verbose= 1,
                                                        class_weight= 'balanced',
                                                        n_jobs = -1
                                                    )
                        t0 = time()
                        clf.fit(train_x, train_y)
                        print("done in %0.3fs." % (time() - t0))
                        predict_y = clf.predict(valid_x)
                        # predict_prob_y = clf.predict_proba(valid_x)
                        # 评价指标值
                        accuracy, precision_avg, recall_avg, f1_score_avg = eval_p_r_f1(valid_y, predict_y)
                        print('accuracy：%0.5f' % accuracy)
                        print('precision_avg: %0.5f' % precision_avg)
                        print('recall_avg: %0.5f' % recall_avg)
                        print('f1_score_avg: %0.5f' % f1_score_avg)
                
                

n_estimator = 1000
max_depth = 10
min_samples_split = 2
min_samples_leaf = 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.7s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s


done in 8.224s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed:    0.3s finished


accuracy：0.50973
precision_avg: 0.64611
recall_avg: 0.50973
f1_score_avg: 0.55530
n_estimator = 1000
max_depth = 10
min_samples_split = 10
min_samples_leaf = 1


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done 1210 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done 1760 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 2410 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 3160 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 4010 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 4960 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 6010 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 7160 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 8410 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 9760 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 10000 out of

done in 451.081s.


[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 3160 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 4960 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 6010 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 7160 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 8410 tasks      | elapsed:    2.9s


accuracy：0.59887
precision_avg: 0.65616
recall_avg: 0.59887
f1_score_avg: 0.61894


[Parallel(n_jobs=20)]: Done 9760 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 10000 out of 10000 | elapsed:    3.4s finished
